In [1]:
import pandas as pd
from time import sleep
from tqdm import tqdm
#from itertools import compress

In [2]:
TARGET_YEAR = '2016'

In [3]:
def check_YYYY(YYYY):
    is_good = isinstance(YYYY, str) and len(YYYY) == 4 and YYYY.isdigit()
    if(not(is_good)):
       raise Exception("Bad args")
    return(is_good)

In [4]:
def clean_names(old_cols):
    new_cols = [c.lower().replace('#', '') for c in old_cols]
    return(new_cols)

In [5]:
def get_events(YYYY):
    check_YYYY(YYYY)
    url = f"https://rusa.org/cgi-bin/resultsearch_PF.pl?regid=&date={YYYY}&type=&dist=&rtid=&esortby=cert&collapse=0"
    print(url)
    
    # get versions with and without links
    dat_links = pd.read_html(url, extract_links='all')[0]
    dat = pd.read_html(url)[0]
    
    # break apart tuples to get urls. we'll have columns called 0 and 1
        # TODO: validate it contains Route and select columns
    route_ids = pd.DataFrame(dat_links[('Route', None)].tolist(), 
                             index=dat_links.index)
    event_ids = pd.DataFrame(dat_links[('select', None)].tolist(), 
                             index=dat_links.index)

    # extract ids from link urls in column 1
        # TODO validate regex, in case format changes
    dat['rt_url'] = route_ids[1]
    dat['rtid'] = route_ids[1].str.extract('^.*rtid=(.*)')
    dat['event_url'] = event_ids[1]
    dat['eid'] = event_ids[1].str.extract('^.*eid=(.*)&')
    
    # clean col names
    new_cols = [c.lower() for c in list(dat.columns)]
    dat.columns = new_cols

    # drop first row and the 'select' column
    dat = dat.drop(0, axis=0)
    dat = dat.drop('select', axis=1)

    return(dat)

In [6]:
events = get_events(TARGET_YEAR)
events.head()

https://rusa.org/cgi-bin/resultsearch_PF.pl?regid=&date=2016&type=&dist=&rtid=&esortby=cert&collapse=0


,region,club,type,distance,date,route,rt_url,rtid,event_url,eid
1,OR: Portland,Oregon Randonneurs / 937020,RUSAP,109,2016/01/01,Wine Country,/cgi-bin/routesearch_PF.pl?rtid=53,53,/cgi-bin/resultsearch_PF.pl?eid=7628&esortby=cert,7628
2,TX: Dallas,Lone Star Randonneurs / 943026,RUSAP,100,2016/01/01,The Doctor's Daughter 100,/cgi-bin/routesearch_PF.pl?rtid=1491,1491,/cgi-bin/resultsearch_PF.pl?eid=7317&esortby=cert,7317
3,GA: Atlanta,Audax Atlanta / 910004,ACPB,200,2016/01/01,Athens Georgia 200k,/cgi-bin/routesearch_PF.pl?rtid=1141,1141,/cgi-bin/resultsearch_PF.pl?eid=7613&esortby=cert,7613
4,OR: Portland,Oregon Randonneurs / 937020,ACPB,200,2016/01/01,Wine Country 200,/cgi-bin/routesearch_PF.pl?rtid=54,54,/cgi-bin/resultsearch_PF.pl?eid=7629&esortby=cert,7629
5,CA: Davis,Davis Bike Club / 905014,ACPB,200,2016/01/01,Davis - Pope Valley - Davis,/cgi-bin/routesearch_PF.pl?rtid=1200,1200,/cgi-bin/resultsearch_PF.pl?eid=7432&esortby=cert,7432


In [7]:
def get_event_details_results(eid):
    
    event_url = f"https://rusa.org/cgi-bin/resultsearch_PF.pl?eid={eid}&esortby=cert"
    #print(event_url)
    two_dats = pd.read_html(event_url)
    
    event_details = two_dats[0]
    event_details.columns = clean_names(list(event_details.columns))
    event_details['eid'] = eid

    # warn if we only get one table back
    if(len(two_dats)<2):
        #print(f'Warning: no finisher results for event {eid}!')
        event_results = pd.DataFrame()
    else:
        event_results = two_dats[1]
        event_results.columns = clean_names(list(event_results.columns))
        # drop the rows 'x nonmembers also finished'
        try:
            event_results = event_results.drop(
                event_results[event_results.cert.str.contains('also finished')].index
            )
        except:
            pass
            #print(f"eid {eid} - couldn't drop 'non member' result rows")
        event_results['eid'] = eid
    
    return([event_details,event_results])

In [8]:
events.groupby('type').size()

type
ACPB     484
ACPF      19
RM         6
RUSAB    114
RUSAF     16
RUSAP    123
dtype: int64

In [9]:
# https://rusa.org/pages/eventtypes
non_team_types = ['ACPB', 'RM', 'RUSAB', 'RUSAP']

In [10]:
# list of event ids for the non team event types, for which we'll get results/details
event_ids = events[events.type.isin(non_team_types)].eid.to_list()
print(len(event_ids))

727


In [11]:
e_detail_list = []
e_results_list = []

In [12]:
for e in tqdm(event_ids):
    #print(e)
    this_d_e = get_event_details_results(e)
    e_detail_list.append(this_d_e[0])
    e_results_list.append(this_d_e[1])
    # sleep roughly every nth call (assuming event ids are consecutive ints)
    if(int(e) % 10 == 0):
        #print('Sleeping')
        sleep(.5)

100%|█████████████████████████████████████████| 727/727 [05:43<00:00,  2.12it/s]


In [13]:
# should all be the same
print(len(e_detail_list), len(e_results_list), len(event_ids))

727 727 727


In [14]:
event_results = pd.concat(e_results_list)

In [15]:
event_details = pd.concat(e_detail_list)

In [16]:
# for when i forgot to filter team events before getting results; 
# the format is different and messes stuff up, plus i don't care about those results right now
# erl2 = list(compress(e_results_list, 
#                      events.type.isin(non_team_types)))

In [17]:
event_details.reset_index(inplace=True, drop=True)
event_results.reset_index(inplace=True, drop=True)

In [18]:
event_results.head()

,cert,rusa,name,club / acp code,time,medal,eid
0,RUSA-P10907,5511,"HUGHART, Holden",Oregon Randonneurs / 937020,06:15,NaN,7628
1,RUSA-P10908,10910,"JACOBSEN, J W",Randonneurs USA / 947095,05:30,NaN,7628
2,RUSA-P10909,1795,"KRAMER, John C",Oregon Randonneurs / 937020,05:22,NaN,7628
3,RUSA-P10910,7729,"LOIBNER, Dieter",Oregon Randonneurs / 937020,04:57,NaN,7628
4,RUSA-P10911,3481,"PETERSON, Scott",Oregon Randonneurs / 937020,05:00,NaN,7628


In [19]:
event_details.head()

,region,club,type,distance,date,finishers,dnf,eid
0,OR: Portland,Oregon Randonneurs / 937020,RUSAP,109,2016/01/01,5,1.0,7628
1,TX: Dallas,Lone Star Randonneurs / 943026,RUSAP,100,2016/01/01,6,0.0,7317
2,GA: Atlanta,Audax Atlanta / 910004,ACPB,200,2016/01/01,23,0.0,7613
3,OR: Portland,Oregon Randonneurs / 937020,ACPB,200,2016/01/01,4,0.0,7629
4,CA: Davis,Davis Bike Club / 905014,ACPB,200,2016/01/01,19,1.0,7432


In [20]:
event_details.to_csv(f'data/event_details_{TARGET_YEAR}.csv', index=False)

In [21]:
event_results.to_csv(f'data/event_results_{TARGET_YEAR}.csv', index=False)

In [22]:
events.to_csv(f'data/events_{TARGET_YEAR}.csv', index=False)